In [48]:
import pandas as pd

In [49]:
df=pd.read_csv('IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, BertTokenizer

#  Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#  Define a custom Keras Layer that returns ONLY the last_hidden_state tensor
class BertLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = TFAutoModel.from_pretrained('bert-base-uncased')

    def call(self, inputs):
        outputs = self.bert(inputs)
        return outputs.last_hidden_state  #  Return only the Tensor (shape: [batch_size, 512, 768])

#  Define input layers for input IDs and attention masks
input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name="attention_mask")

#  Use the custom BERT layer
bert_output = BertLayer()({
    'input_ids': input_ids,
    'attention_mask': attention_mask
})  # This now returns a Tensor

#  Extract the [CLS] token's representation
cls_output = tf.keras.layers.Lambda(lambda x: x[:, 0, :])(bert_output)  # Shape: (batch_size, hidden_size)

#  Add dropout and dense layer for binary classification
x = tf.keras.layers.Dropout(0.1)(cls_output)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

#  Build the complete model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

#  Print model summary
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ attention_mask      │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_ids           │ (None, 512)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_layer_2        │ (None, 512, 768)  │          0 │ attention_mask[0… │
│ (BertLayer)         │                   │            │ input_ids[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 768)       │          0 │ bert_layer_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 768)       │          0 │ lambda_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │        769 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

In [51]:
X = df['review']                        # the text column
y = df['sentiment'].map({'positive': 1, 'negative': 0})  # convert labels to binary

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:

#  Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 🔹 Tokenize train set
train_encoding = tokenizer(
    X_train.tolist(),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='tf'
)

# 🔹 Tokenize test set
test_encoding = tokenizer(
    X_test.tolist(),
    padding='max_length',
    truncation=True,
    max_length=512,
    return_tensors='tf'
)


In [58]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # ✅ Use this if labels are integers
    metrics=['accuracy']
)

history = model.fit(
    x={
        'input_ids': train_encoding['input_ids'],
        'attention_mask': train_encoding['attention_mask']
    },
    y=y_train,
    validation_data=(
        {
            'input_ids': test_encoding['input_ids'],
            'attention_mask': test_encoding['attention_mask']
        },
        y_test
    ),
    epochs=3,
    batch_size=16
)



Epoch 1/3
69/69 ━━━━━━━━━━━━━━━━━━━━ 2536s 37s/step - accuracy: 0.5501 - loss: 0.6922 - val_accuracy: 0.7826 - val_loss: 0.5632
Epoch 2/3
69/69 ━━━━━━━━━━━━━━━━━━━━ 1246s 18s/step - accuracy: 0.7520 - loss: 0.5583 - val_accuracy: 0.8442 - val_loss: 0.4811
Epoch 3/3
69/69 ━━━━━━━━━━━━━━━━━━━━ 1043s 15s/step - accuracy: 0.7814 - loss: 0.5077 - val_accuracy: 0.8514 - val_loss: 0.4347


In [59]:
predictions = model.predict({
    'input_ids': test_encoding['input_ids'],
    'attention_mask': test_encoding['attention_mask']
})

9/9 ━━━━━━━━━━━━━━━━━━━━ 239s 25s/step


In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted_labels))

              precision    recall  f1-score   support

           0       0.36      0.16      0.22       133
           1       0.49      0.74      0.59       143

    accuracy                           0.46       276
   macro avg       0.42      0.45      0.40       276
weighted avg       0.43      0.46      0.41       276



In [62]:
text = "The movie was a complete waste of time. The plot was boring, the acting was terrible, and I couldn't wait for it to end."

# Tokenize
new_encoding = tokenizer(
    text,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='tf'
)

# Predict
prediction = model.predict({
    'input_ids': new_encoding['input_ids'],
    'attention_mask': new_encoding['attention_mask']
})

# Get class
predicted_class = (prediction > 0.5).astype(int).flatten()

# Map to label
label = "Positive" if predicted_class[0] == 1 else "Negative"

print("Review text:", text)
print("Predicted sentiment:", label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
Review text: The movie was a complete waste of time. The plot was boring, the acting was terrible, and I couldn't wait for it to end.
Predicted sentiment: Negative
